# Instalar y configurar PostgreSQL - EICU-CRD

## Instalar PostgreSQL según documentación oficial

Ingresar a https://www.postgresql.org/download/ para detalles sobre la instalación en diferetes SO.

Para confirmar la instalación, ejecutar en el terminal:

```
sudo service postgresql start
psql --version
```

## Configurar contraseñas de usuario

Es necesario configurar la contraseña del usuario por defecto (`postgres`) para continuar sin problemas
con la instalación de EICU-CRD.

Primero, entrar a la consola de PostgreSQL: `sudo -u postgres psql`.
Una vez dentro, cambiar la contraseña del usuario ejecutando el comando `\password`. Se recomienda cambiar la contraseña al mismo nombre del usuario: `postgres`. Para salir de la consola, ejecutar `\q`.

En segundo lugar, verificar que postgres exige autenticación por contraseña. Para esto, abrir con un editor de texto el archivo `pg_hab.conf`. La ruta al archivo puede variar ligeramente dependiendo de la versión de PostgreSQL. En este caso se muestra un comando para acceder al archivo con PostgreSQL 13, utilizando Nano como editor de texto:

```sudo nano /etc/postgresql/13/main/pg_hba.conf```.

Casi al final del archivo se encontrará el siguiente texto:

```
# Database administrative login by Unix domain socket
local    all    postgres    md5
```

En el caso de que diga `local    all    postgres    peer`, cambiar `peer` a `md5` y reiniciar el servicio con el comando `sudo service postgresql restart`.

Por último, intentar acceder a la consola de PostgreSQL nuevamente, indicando la contraseña: `sudo -u postgres psql`.

## Instalar EICU-CRD

Descargar y descomprimir los archivos CSV de la base de datos EICU-CRD, para esto es necesario obtener licencia de PhysioNet, en caso de no tnerla, se puede utilizar el demo: https://physionet.org/static/published-projects/eicu-crd-demo/eicu-collaborative-research-database-demo-2.0.zip.

En la carpeta `eicu-code` está el repositorio de eICU-CRD utilizado para construir la base de datos en PostgreSQL. Sin embargo, es posible clonar el repositorio `eicu-code`: `git clone https://github.com/MIT-LCP/eicu-code.git`. Procurar que el espacio de trabajo quede con una estructura como la siguiente:
```
.
├── data
│   └── eicu                        // Archivos CSV con las tablas de la base de datos
│       ├── admissiondrug.csv              
│       └── ...
│
├── eicu-code                       // Repositorio MIMIC
│   ├── build-db                    // Scripts para construir EICU-CRD en diferentes DBMS
│   ├── concepts                    // Vistas SQL y otras utilidades
│   ├── notebooks                   // Notebooks de Jupyter y otros ejemplos de uso
│   └── ...
└── ...
```

Para construir la base de datos EICU-CRD, abrir un terminal en `eicu-code/build-db/postgres` y ejectuar el makefile presente: `make eicu datadir="../../../data/eicu/"`. Reemplazar `datadir` con la ruta pertinente hacia la carpeta con los archivos CSV en caso de que la propuesta no sea correcta.

El makefile creara la base de datos y esquema `eicu`. Para verificar la instalación y entrar a la base de datos, ejecutar en el terminal: `psql eicu postgres`.

## Entorno de Python

Los ejemplos de uso de este notebook se hicieron utilizando Python 3.8, a través de un entorno configurando
por Anaconda. Las dependencias (hasta ahora) son:

+ Psycopg2: Adaptador de PostgreSQL para Python.
+ Pandas: Libraría de manipulación de tablas y series de datos.

Para descargar e installar Anaconda, visitar: https://www.anaconda.com/products/individual#Downloads.

Para crear el ambiente virtual delproyecto, ejecutar: `conda create -n aidxmods python>=3.8`.

Para activar el ambiente recién creado: `conda activate aidxmods`.

Para instalar las dependencias: `conda install -n aidxmods pandas psycopg2`. Para instalar paquetes adicionales se puede utilizar el mismo comando. **NO UTILIZAR PIP PARA INSTALAR PAQUETES, SÓLO UTILIZAR CONDA**.

# Conectar con la base de datos

Si la instalación ha sido correcta, se podrá contar el total de pacientes en la base de datos.

In [1]:
import psycopg2 as pg
import pandas as pd

# Crear conexión a la base de datos
connection = pg.connect(user='postgres', password='postgres', host='localhost', dbname='eicu')

# Set schema
cursor = connection.cursor()
cursor.execute('SET search_path TO public,eicu_crd')
connection.commit()
cursor.close()

# Comprobar conexión con una consulta de prueba
query = "SELECT count(*) FROM patient"
pd.read_sql(query, connection)

,count
0,200859


# Creación de tablas de eICU-CRD

Ahora se crearán las tablas que alimentarán al análisis principal del proyecto.

In [2]:
import psycopg2 as pg
import psycopg2.sql as sql
import pandas as pd

In [3]:
def make_table(schema, table, connection):
    '''Creación de tablas.'''
    
    path = f'sql/{schema}/{table}.sql'
    
    # Obtener el SQL para crear la vista
    try:
        print(f'Abriendo el archivo "{path}"')
        query = open(path, 'r').read()
        
    except Exception as error:
        print(f'No se pudo abrir el archivo "{path}":\n>>> {error}')
        return

    # Crear la tabla
    try:
        print(f'Creando la tabla {schema}.{table}')
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        cursor.close()
        
        print(f'Se creó la tabla {schema}.{table} exitósamente')
    
    except Exception as error:
        print(f'No se pudo crear la tabla {schema}.{table}:\n>>> {error}')
        connection.commit()
        cursor.close()
    
    # Solicitar recuento
    try:
        cursor = connection.cursor()
        query = sql.SQL("select count(*) from {schema}.{table}").format(
            schema=sql.Identifier(schema),
            table=sql.Identifier(table)
        )
        cursor.execute(query)
        count = cursor.fetchone()[0]
        connection.commit()
        cursor.close()
        
        print('SELECT', count)
    
    except Exception as error:
        print(f'No se pudo solicitar recuento de la tabla {schema}.{table}:\n>>> {error}')
        connection.commit()
        cursor.close()
    
    return

# Enlistar tablas a crear
tables = [
    'charttimes',
    'admissions',
    'diagnoses',
    'gcs',
    'heightweights',
    'inputs',
    'labs',
    'outputs',
    'vitals',
]

In [4]:
# Crear conexión a la base de datos
connection = pg.connect(user='postgres', password='postgres', host='localhost', dbname='eicu')

# Set schema
for table in tables:
    make_table('aidxmods', table, connection)
else:
    make_table('aidxmods', 'firstday', connection)

print('Proceso completado')
connection.close()

Abriendo el archivo "sql/aidxmods/charttimes.sql"
Creando la tabla aidxmods.charttimes
Se creó la tabla aidxmods.charttimes exitósamente
SELECT 95555
Abriendo el archivo "sql/aidxmods/admissions.sql"
Creando la tabla aidxmods.admissions
Se creó la tabla aidxmods.admissions exitósamente
SELECT 95555
Abriendo el archivo "sql/aidxmods/diagnoses.sql"
Creando la tabla aidxmods.diagnoses
Se creó la tabla aidxmods.diagnoses exitósamente
SELECT 95555
Abriendo el archivo "sql/aidxmods/gcs.sql"
Creando la tabla aidxmods.gcs
Se creó la tabla aidxmods.gcs exitósamente
SELECT 95555
Abriendo el archivo "sql/aidxmods/heightweights.sql"
Creando la tabla aidxmods.heightweights
Se creó la tabla aidxmods.heightweights exitósamente
SELECT 95555
Abriendo el archivo "sql/aidxmods/inputs.sql"
Creando la tabla aidxmods.inputs
Se creó la tabla aidxmods.inputs exitósamente
SELECT 95555
Abriendo el archivo "sql/aidxmods/labs.sql"
Creando la tabla aidxmods.labs
Se creó la tabla aidxmods.labs exitósamente
SELECT 9